In [1]:
import tensorflow.lite as tflite
from keras_image_helper import create_preprocessor
import pickle
import numpy as np

preprocessor = create_preprocessor('xception', target_size=(224, 224))


2022-12-20 22:17:43.100702: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-20 22:17:43.571128: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-20 22:17:43.571151: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-20 22:17:45.044669: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-

In [2]:
interpreter = tflite.Interpreter(model_path='model_0.936.tflite')
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [3]:
with open("classes", "rb") as fp:   # Unpickling
    classes = pickle.load(fp)

['Afghan',
 'African Wild Dog',
 'Airedale',
 'American  Spaniel',
 'American Hairless',
 'Basenji',
 'Basset',
 'Beagle',
 'Bearded Collie',
 'Bermaise',
 'Bichon Frise',
 'Blenheim',
 'Bloodhound',
 'Bluetick',
 'Border Collie',
 'Borzoi',
 'Boston Terrier',
 'Boxer',
 'Bull Mastiff',
 'Bull Terrier',
 'Bulldog',
 'Cairn',
 'Chihuahua',
 'Chinese Crested',
 'Chow',
 'Clumber',
 'Cockapoo',
 'Cocker',
 'Collie',
 'Corgi',
 'Coyote',
 'Dalmation',
 'Dhole',
 'Dingo',
 'Doberman',
 'Elk Hound',
 'French Bulldog',
 'German Sheperd',
 'Golden Retriever',
 'Great Dane',
 'Great Perenees',
 'Greyhound',
 'Groenendael',
 'Irish Spaniel',
 'Irish Wolfhound',
 'Japanese Spaniel',
 'Komondor',
 'Labradoodle',
 'Labrador',
 'Lhasa',
 'Malinois',
 'Maltese',
 'Mex Hairless',
 'Newfoundland',
 'Pekinese',
 'Pit Bull',
 'Pomeranian',
 'Poodle',
 'Pug',
 'Rhodesian',
 'Rottweiler',
 'Saint Bernard',
 'Schnauzer',
 'Scotch Terrier',
 'Shar_Pei',
 'Shiba Inu',
 'Shih-Tzu',
 'Siberian Husky',
 'Vizsla'

In [12]:
# url = 'https://glorypets.ru/wp-content/uploads/2020/07/1-tsarstvennost.jpg'
url = 'https://sun9-79.userapi.com/c11422/u1430261/148960630/x_b6d7669e.jpg'

def predict(url):
    X = preprocessor.from_url(url)

    interpreter.set_tensor(input_index, X)
    interpreter.invoke()
    preds = interpreter.get_tensor(output_index)

    ind = np.argpartition(preds[0], -2)[-2:]
    # print('ind', ind)
    # print(np.diff(preds[0][ind]))
    if np.diff(preds[0][ind]) > 0.8: # if clear the breed
        return {classes[np.argmax(preds[0])]: np.max(preds).tolist()}
    else: # half breed
        h_breed = {}
        for i in ind:
            print(preds[0][i], 'preds[0][i]')
            h_breed[classes[i]]=preds[0][i].tolist()
        # h_breed = dict(zip(classes, preds[0]))
        return h_breed


In [13]:
predict(url)

0.24571152 preds[0][i]
0.71995497 preds[0][i]


{'Rottweiler': 0.2457115203142166, 'Doberman': 0.7199549674987793}